In [ ]:
from tensorflow.keras.models import load_model
import os

%load_ext autoreload 
%autoreload 2

import tensorflow as tf
from tensorflow.keras import layers, models


# From model script, workaround to serialize embed layer
@tf.keras.utils.register_keras_serializable()
class L2Normalization(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(L2Normalization, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.math.l2_normalize(inputs, axis=1)

    def get_config(self):
        config = super(L2Normalization, self).get_config()
        return config


# Load the trained encoder
# encoder = load_model("../drawing_encoder_model.h5")
encoder = tf.keras.models.load_model(
    "../drawing_encoder_model.h5",
    custom_objects={'L2Normalization': L2Normalization}
)


In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from scipy.spatial.distance import cdist
from raw_to_dataset import sample_points_fixed, load_drawing_xyz

# Computes the distance matrix for a list of points

def compute_distance_matrix(points):
    """
    points: (num_points, 3)
    returns: (num_points, num_points) distance matrix
    """
    points = np.asarray(points, dtype=np.float32)

    diff = points[:, None, :] - points[None, :, :]
    dist = np.linalg.norm(diff, axis=-1)

    scale = np.max(dist)
    return dist / (scale + 1e-8)


def compute_distance_matrix_heightchannel(points):
    """
    points: (num_points, 3)
    returns: (num_points, num_points, 2) -> [Distance, Y-Diff]
    """
    points = np.asarray(points, dtype=np.float32)

    # Channel 1: Euclidean Distance (The Shape)
    diff = points[:, None, :] - points[None, :, :]
    dist = np.linalg.norm(diff, axis=-1)
    
    # Global scale for normalization (applied to both channels to keep aspect ratio)
    scale = np.max(dist) + 1e-8
    dist_norm = dist / scale

    # Channel 2: Y-Axis Difference (The Orientation / Up-ness)
    # Calculates (y1 - y2) for every point pair
    y_diff = points[:, 1][:, None] - points[None, :, 1]
    y_diff_norm = y_diff / scale

    # Stack them: Result is (128, 128, 2)
    return np.stack([dist_norm, y_diff_norm], axis=-1)


# Gets an embedding for a drawing (represented by a list of xyz points)
def get_embedding(drawing_points):
    """
    drawing_points: (128, 3) numpy array of a drawing
    returns: (EMBED_DIM,) embedding
    """
    dist_matrix = compute_distance_matrix_heightchannel(drawing_points)
    dist_matrix = np.expand_dims(dist_matrix, axis=0)  # batch dimension
    embedding = encoder.predict(dist_matrix, verbose=0)
    return embedding[0]



# Gets a comparison score between 2 drawings
def compare_drawings(drawing1, drawing2):
    """
    drawing1: (128, 3) numpy array of a drawing
    drawing2: (128, 3) numpy array of a drawing
    """
    emb1 = get_embedding(drawing1)
    emb2 = get_embedding(drawing2)

    score = np.linalg.norm(emb1 - emb2)
    return score

# Gets comparison score between 2 drawings, given filepaths to .json representations
def compare_drawings_filepath(filepath1, filepath2):
    """
    filepath1, filepath2: string filepaths to .json format of drawings, can be of any size. See sample data.
    """
    points1 = load_drawing_xyz(filepath1)
    points2 = load_drawing_xyz(filepath2)
    return compare_drawings(points1, points2)



In [3]:
# Compare to all .json drawings in sample_data, 
#  - allows multiple representations of each drawing and takes the average score
base_dir = "../dataset_RAW/validation"

# Returns the name of the closest drawing folder to a specified .json file
def get_closest_class_folder(filename, list_top_N=5):
    lowest_score = float('inf') # lower = closer match
    match = "NOT FOUND, CHECK base_dir OR FOLDER LAYOUT"
    chosen_file = "NONE"
    scores = []

    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        if not os.path.isdir(folder_path):
            continue
        
        for file_to_compare in os.listdir(folder_path):
            # Skip same file or invalid
            comparison_drawing_path = os.path.join(folder_path, file_to_compare)

            if comparison_drawing_path == filename or not file_to_compare.endswith(".json"):
                continue

            dist = compare_drawings_filepath(filename, comparison_drawing_path)
            if (dist < lowest_score):
                lowest_score = dist
                match = folder
                chosen_file = file_to_compare

            scores.append((dist, file_to_compare))

    scores.sort()
    print(f"\t\tGot top 5: {scores[0:5]}")
    return match, chosen_file
          
num_correct = 0
num_drawings = 0
drawing_scores = {} # dict to view accuracy for each drawing

# Get accuracy of comparison between every drawing class' examples, using the validation dataset as representations for each class
for folder in os.listdir(base_dir):
    print(f"Comparing {folder}...")
    folder_path = os.path.join(base_dir, folder)

    if not os.path.isdir(folder_path):
        continue
    
    num_correct_here = 0
    # Get score of each drawing
    for i, filename in enumerate(os.listdir(folder_path)):
        if not filename.endswith(".json"):
            continue
        print(f"\tFinding match to {filename}...")

        drawing_path = os.path.join(folder_path, filename)
        
        match, chosen_file = get_closest_class_folder(drawing_path)

        if (match == folder):
            num_correct_here += 1
            print(f"\t\tCorrect match to {filename} [Score: {num_correct_here}/{i + 1}] (chose {chosen_file})")
        else:
            print(f"\t\tIncorrect match to {filename} [Score: {num_correct_here}/{i + 1}] (chose {chosen_file})")

    num_drawings_here = len(os.listdir(folder_path))
    drawing_scores[folder] = f"{num_correct_here}/{num_drawings_here}"

    num_correct += num_correct_here
    num_drawings += num_drawings_here

# Print results for each drawing
for folder, score in drawing_scores.items():
    if folder is None:
        print(f"{folder}: (no drawings found)")
    else:
        print(f"{folder}: {score}")

print(f"Completed Validation. Overall Score: {num_correct / num_drawings} ({num_correct}/{num_drawings})")

Comparing cube...
	Finding match to cube_20251231_170814_696.json...
		Got top 5: [(0.26297984, 'cube_20251231_171503_880.json'), (0.3632137, 'cube_20251231_170949_256.json'), (0.3803306, 'triPrism_vertical_20260101_135552_977.json'), (0.42378822, 'cube_20251231_171225_854.json'), (0.4475908, 'triPrism_vertical_20260101_135503_462.json')]
		Correct match to cube_20251231_170814_696.json [Score: 1/1] (chose cube_20251231_171503_880.json)
	Finding match to cube_20251231_170949_256.json...
		Got top 5: [(0.10817425, 'cube_20251231_171225_854.json'), (0.22735706, 'rectangle_horizontal_20251231_172850_073.json'), (0.2340456, 'cube_20251231_171503_880.json'), (0.3632137, 'cube_20251231_170814_696.json'), (0.37512967, 'rectangle_horizontal_20251231_173231_647.json')]
		Correct match to cube_20251231_170949_256.json [Score: 2/2] (chose cube_20251231_171225_854.json)
	Finding match to cube_20251231_171225_854.json...
		Got top 5: [(0.10817425, 'cube_20251231_170949_256.json'), (0.2258948, 'cube

In [ ]:
# Compare to all .json drawings in sample_data, 
#  - allows multiple representations of each drawing and takes the average score
base_dir = "sample_data"
reference_filepath = "sample_data/concert_sample/concert_sample_default.json"
# reference_filepath = "sample_data/apt_verticalL_sample/apt_verticalL_sample_default3.json"

avg_scores = {} # Dictionary to store average score for each drawing

# Find average of comparisons to each drawing class' examples
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)

    if not os.path.isdir(folder_path):
        continue

    scores = []

    # Get score of each drawing
    for filename in os.listdir(folder_path):
        if not filename.endswith(".json"):
            continue

        drawing_path = os.path.join(folder_path, filename)

        # Skip the reference file itself in case using sample data as example
        if drawing_path == reference_filepath:
            continue

        dist = compare_drawings_filepath(reference_filepath, drawing_path)
        scores.append(dist)

    if scores:
        avg_scores[folder] = sum(scores) / len(scores)
    else:
        avg_scores[folder] = None  # no valid files

# Print results
for folder, avg in avg_scores.items():
    if avg is None:
        print(f"{folder}: (no drawings found)")
    else:
        print(f"{folder}: {avg:.4f}")

apt1_sample: 0.5412
apt_verticalL_sample: 0.2757
concert_sample: 0.5281
house1_sample: 1.0279


In [5]:
import numpy as np
def compute_distance_matrix_N(points):
    """
    points: (num_points, 3)
    returns: (num_points, num_points) distance matrix
    """
    points = np.asarray(points, dtype=np.float32)
    diff = points[:, np.newaxis, :] - points[np.newaxis, :, :]
    dist_matrix = np.linalg.norm(diff, axis=-1)
    return dist_matrix


def compute_distance_matrix_relative(points):
    """
    points: (num_points, 3)
    returns: (num_points, num_points) distance matrix
    """
    points = np.asarray(points, dtype=np.float32)

    diff = points[:, None, :] - points[None, :, :]
    dist = np.linalg.norm(diff, axis=-1)

    scale = np.max(dist)
    return dist / (scale + 1e-8)


points_ex = [[1, 2, 3], [2, 2, 3], [2, 3, 3], [1, 3, 3]]
dist_matrix = compute_distance_matrix_N(points_ex)
dist_matrix_relative = compute_distance_matrix_relative(points_ex)
print(f"DistMatrix (Absolute):\n{dist_matrix}")
print(f"DistMatrix (Relative):\n{dist_matrix_relative}")


DistMatrix (Absolute):
[[0.        1.        1.4142135 1.       ]
 [1.        0.        1.        1.4142135]
 [1.4142135 1.        0.        1.       ]
 [1.        1.4142135 1.        0.       ]]
DistMatrix (Relative):
[[0.         0.70710677 1.         0.70710677]
 [0.70710677 0.         0.70710677 1.        ]
 [1.         0.70710677 0.         0.70710677]
 [0.70710677 1.         0.70710677 0.        ]]
